In [ ]:
# 启动SAP GUI、连接服务器并执行报表下载
import os
import subprocess
import time
import win32com.client
import sys

# 强制关闭 Excel 的函数
def close_excel():
    """强制关闭所有 Excel 进程"""
    try:
        result = subprocess.run(
            ["taskkill", "/F", "/IM", "EXCEL.EXE"],
            capture_output=True,
            timeout=5,
            text=True
        )
        if "成功" in result.stdout or "SUCCESS" in result.stdout:
            print("✓ 已关闭 Excel")
            return True
        elif "找不到" in result.stdout or "not found" in result.stdout.lower():
            print("✓ Excel 未运行")
            return True
        else:
            return False
    except Exception as e:
        print(f"⚠ 关闭 Excel 时出错: {e}")
        return False

# 多个可能的SAP GUI路径
sap_paths = [
    r"C:\ProgramData\Microsoft\Windows\Start Menu\Programs\SAP Front End\SAP Logon Pad.lnk",
    r"C:\Program Files (x86)\SAP\FrontEnd\SAPgui\saplogon.exe",
    r"C:\Program Files\SAP\FrontEnd\SAPgui\saplogon.exe",
    r"C:\ProgramData\Microsoft\Windows\Start Menu\Programs\SAP Front End.sap",
]

try:
    # 检查SAP GUI是否已经运行
    try:
        SapGuiAuto = win32com.client.GetObject("SAPGUI")
        print("✓ SAP GUI 已在运行")
    except:
        print("SAP GUI 未运行，正在查找启动文件...")
        
        # 尝试每个路径
        found = False
        for sap_path in sap_paths:
            if os.path.exists(sap_path):
                print(f"找到: {sap_path}")
                print("正在启动 SAP GUI...")
                subprocess.Popen([sap_path], shell=True)
                print("SAP GUI 正在启动...")
                time.sleep(5)
                found = True
                break
        
        if not found:
            print("✗ 未找到SAP GUI启动文件")
            print("\n已尝试以下路径:")
            for path in sap_paths:
                print(f"  - {path}")
            print("\n请手动启动SAP GUI并登录")
            sys.exit(1)
        
        # 重新获取SAP GUI对象
        try:
            SapGuiAuto = win32com.client.GetObject("SAPGUI")
        except:
            print("✗ SAP GUI 启动失败")
            sys.exit(1)
    
    # 检查是否已有连接
    application = SapGuiAuto.GetScriptingEngine
    
    if application.Children.Count > 0:
        connection = application.Children(0)
        if connection.Children.Count > 0:
            session = connection.Children(0)
            print(f"✓ 已登录 - 用户: {session.Info.User}, 系统: {session.Info.SystemName}")
        else:
            print("⚠ 有连接但未登录，请手动登录")
            sys.exit(1)
    else:
        print("\n正在连接到 RPR - ECC Production 服务器...")
        
        # 获取SAP Logon窗口并连接
        try:
            connection = application.OpenConnection("RPR - ECC Production", True)
            print("✓ 已发起连接到 RPR - ECC Production")
            print("请在弹出的登录窗口中输入用户名和密码...")
            
            # 等待用户登录（最多等待60秒）
            for i in range(60):
                time.sleep(1)
                if connection.Children.Count > 0:
                    session = connection.Children(0)
                    print(f"\n✓✓✓ 登录成功！")
                    print(f"用户: {session.Info.User}")
                    print(f"系统: {session.Info.SystemName}")
                    print(f"客户端: {session.Info.Client}")
                    break
                if (i + 1) % 10 == 0:
                    print(f"等待登录中... ({i + 1}秒)")
            else:
                print("⚠ 等待超时，请手动登录后重新运行")
                sys.exit(1)
                
        except Exception as e:
            print(f"✗ 连接服务器失败: {e}")
            print("\n请手动操作：")
            print("1. 在SAP Logon窗口中双击 'RPR - ECC Production'")
            print("2. 输入用户名和密码登录")
            sys.exit(1)
    
    print("\n" + "="*60)
    print("开始执行报表下载自动化")
    print("="*60)
    
    # 最大化窗口
    session.findById("wnd[0]").maximize()
    time.sleep(0.5)
    print("✓ 窗口已最大化")
    
    # 在命令栏输入事务代码 S_ALR_87011990
    session.findById("wnd[0]/tbar[0]/okcd").text = "S_ALR_87011990"
    session.findById("wnd[0]").sendVKey(0)  # 按回车键执行
    time.sleep(3)
    print("✓ 已进入事务代码 S_ALR_87011990")
    
    # 定义要下载的报表列表
    reports = [
        {"variant": "Z025_01", "filename": "SAP_FA LIST_01_Z025"},
        {"variant": "Z025_05", "filename": "SAP_FA LIST_05_Z025"},
        {"variant": "ZTXT_05", "filename": "SAP_FA LIST_05_ZTXT"},
        {"variant": "Z233_05", "filename": "SAP_FA LIST_05_Z233"},
        {"variant": "Z503_05", "filename": "SAP_FA LIST_05_Z503"}
    ]
    
    # 循环下载每个报表
    for idx, report in enumerate(reports, 1):
        print(f"\n{'='*60}")
        print(f"开始下载第 {idx} 个报表: {report['variant']}")
        print(f"{'='*60}")
        
        # 尝试多种方式输入报表名称
        input_success = False
        
        # 尝试1: 在选择屏幕直接输入
        try:
            session.findById("wnd[0]/usr/ctxtPA_VARI").text = report['variant']
            time.sleep(0.5)
            print(f"✓ 已输入变式 {report['variant']}")
            input_success = True
        except Exception as e1:
            print(f"尝试1失败: 未找到变式输入框")
        
        # 尝试2: 通过F4帮助选择
        if not input_success:
            try:
                # 点击变式帮助按钮
                session.findById("wnd[0]/tbar[1]/btn[17]").press()
                time.sleep(1)
                
                # 在弹出窗口输入
                session.findById("wnd[1]/usr/txtV-LOW").text = report['variant']
                session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 7
                time.sleep(0.5)
                print(f"✓ 已在弹出窗口输入 {report['variant']}")
                
                # 确认选择
                session.findById("wnd[1]/tbar[0]/btn[8]").press()
                time.sleep(1.5)
                print("✓ 已确认选择")
                input_success = True
            except Exception as e2:
                print(f"尝试2失败: {e2}")
        
        # 尝试3: 直接执行（如果已有默认变式）
        if not input_success:
            print("⚠ 无法输入报表名称，尝试直接执行...")
            time.sleep(0.5)
        
        # 设置日期为当月最后一天
        from datetime import datetime
        from calendar import monthrange
        
        # 获取当前年月和月末日期
        now = datetime.now()
        year = now.year
        month = now.month
        last_day = monthrange(year, month)[1]
        end_date = f"{month:02d}/{last_day:02d}/{year:04d}"
        end_date_formatted = f"{month:02d}/{last_day:02d}/{year:04d}"  # SAP 日期格式
        
        print(f"正在设置日期为当月最后一天: {end_date_formatted}")
        
        date_set_success = False
        
        # 方法1: 直接输入日期文本（最简单）
        try:
            session.findById("wnd[0]/usr/ctxtBERDATUM").text = end_date_formatted
            time.sleep(0.3)
            print(f"✓ 已设置日期为: {end_date_formatted} (方法1: 直接输入)")
            date_set_success = True
        except:
            pass  # 静默失败，尝试下一个方法
        
        # 方法2: 尝试其他可能的日期字段 ID
        if not date_set_success:
            try:
                session.findById("wnd[0]/usr/ctxtPA_STIDA").text = end_date_formatted
                time.sleep(0.3)
                print(f"✓ 已设置日期为: {end_date_formatted} (方法2: PA_STIDA)")
                date_set_success = True
            except:
                pass  # 静默失败，尝试下一个方法
        
        # 方法3: 通过日历控件选择
        if not date_set_success:
            try:
                # 设置焦点到日期字段
                session.findById("wnd[0]/usr/ctxtBERDATUM").setFocus()
                session.findById("wnd[0]/usr/ctxtBERDATUM").caretPosition = 10
                
                # 打开日期选择器（F4键）
                session.findById("wnd[0]").sendVKey(4)
                time.sleep(0.5)
                
                # 在日历控件中选择日期
                session.findById("wnd[1]/usr/cntlCONTAINER/shellcont/shell").focusDate = end_date
                session.findById("wnd[1]/usr/cntlCONTAINER/shellcont/shell").selectionInterval = f"{end_date},{end_date}"
                time.sleep(0.5)
                
                # 确认日期选择
                session.findById("wnd[1]/tbar[0]/btn[0]").press()
                time.sleep(0.5)
                
                print(f"✓ 已设置日期为: {end_date_formatted} (方法3: 日历控件)")
                date_set_success = True
            except:
                pass  # 静默失败
        
        # 如果所有方法都失败
        if not date_set_success:
            print(f"ℹ 未找到日期字段，将使用变式中保存的默认日期")
            print(f"  变式 {report['variant']} 应已包含正确的日期参数")
        
        time.sleep(0.5)
        
        # 执行报表
        session.findById("wnd[0]/tbar[1]/btn[8]").press()
        time.sleep(3)
        print("✓ 报表正在执行...")
        
        # 选择导出菜单
        session.findById("wnd[0]/mbar/menu[0]/menu[1]/menu[1]").select()
        time.sleep(1)
        print("✓ 已打开导出菜单")
        
        # 设置文件名
        session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_GUI_CUL_EXPORT_AS:0512/txtGS_EXPORT-FILE_NAME").text = report['filename']
        session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_GUI_CUL_EXPORT_AS:0512/txtGS_EXPORT-FILE_NAME").caretPosition = len(report['filename'])
        time.sleep(0.5)
        print(f"✓ 已设置文件名: {report['filename']}")
        
        # 继续
        session.findById("wnd[1]/tbar[0]/btn[20]").press()
        time.sleep(1)
        
        # 设置保存路径
        session.findById("wnd[1]/usr/ctxtDY_PATH").setFocus()
        session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 0
        session.findById("wnd[1]").sendVKey(4)  # F4 帮助
        time.sleep(0.5)
        
        # 根据当前月份动态生成路径
        month_mapping = {
            11: "02 November",
            12: "03 December",
            1: "04 January",
            2: "05 February",
            3: "06 March",
            4: "07 April",
            5: "08 May",
            6: "09 June",
            7: "10 July",
            8: "11 August",
            9: "12 September"
        }
        
        current_month = datetime.now().month
        month_folder = month_mapping.get(current_month, f"{current_month:02d} Month")
        year_month_str = datetime.now().strftime("%Y%m")
        
        # 输入目标路径
        save_path = rf"\\apac.wdpr.disney.com\Corp\CNKC\Finance\Controllership\FINANCIAL REPORTING\FY_ 2026\Month-End\{month_folder}\FA Ledger\Report\SAP report {year_month_str}"
        session.findById("wnd[2]/usr/ctxtDY_PATH").text = save_path
        session.findById("wnd[2]/usr/ctxtDY_PATH").setFocus()
        session.findById("wnd[2]/usr/ctxtDY_PATH").caretPosition = len(save_path)
        time.sleep(0.5)
        print(f"✓ 已设置保存路径: {month_folder}")
        
        # 关闭路径选择窗口
        session.findById("wnd[2]/tbar[0]/btn[0]").press()
        time.sleep(0.5)
        print(f"保存位置: {save_path}\\{report['filename']}.xlsx")
        # 设置"不替换"选项（防止 Excel 自动打开）
        try:
            # 取消勾选"用合适的应用程序打开"复选框
            session.findById("wnd[1]/usr/chxDY_NO_REPLACE").selected = True
            print("✓ 已设置为后台保存模式（不打开 Excel）")
        except:
            try:
                # 另一种可能的ID
                session.findById("wnd[1]/usr/ctxtDY_NO_REPLACE").selected = True
                print("✓ 已设置为后台保存模式（不打开 Excel）")
            except:
                print("⚠ 无法找到后台保存选项，文件可能会自动打开")
        
        time.sleep(0.5)
        
        # 确认保存，同时开始监控 Excel
        session.findById("wnd[1]/tbar[0]/btn[11]").press()  # 点击"生成"按钮
        
        print(f"\n✓✓✓ 第 {idx} 个报表已开始导出...")
        print(f"保存位置: \\\\apac.wdpr.disney.com\\Corp\\CNKC\\Finance\\Controllership\\FINANCIAL REPORTING\\FY_ 2026\\Month-End\\02 November\\FA Ledger\\Report\\SAP report 202511\\{report['filename']}.xlsx")
        
        # 等待文件保存完成
        time.sleep(3)
        
        # 关闭所有 SAP 弹出窗口
        try:
            while session.Children.Count > 1:
                session.findById("wnd[1]").close()
                time.sleep(0.3)
        except:
            pass
    
        # 返回到事务代码选择屏幕
        try:
            session.findById("wnd[0]/tbar[0]/btn[3]").press()  # 按返回按钮
            time.sleep(0.5)
            print("✓ 已返回到 S_ALR_87011990 选择屏幕")
        except:
            print("⚠ 无法返回选择屏幕")
        
        # 每个报表完成后，关闭可能打开的 Excel
        print(f"\n第 {idx} 个报表完成，检查并关闭 Excel...")
        close_excel()
        time.sleep(1)
        
        # 如果不是最后一个报表，提示准备下一个
        if idx < len(reports):
            print(f"准备下载下一个报表...")
            time.sleep(1)
        
    
    print(f"\n{'='*60}")
    print(f"✓✓✓ 所有任务完成！共下载了 {len(reports)} 个报表")
    print(f"{'='*60}")
    
except Exception as e:
    print(f"\n✗ 发生错误: {e}")
    print(f"错误类型: {type(e).__name__}")
    import traceback
    print("\n详细错误信息:")
    traceback.print_exc()
 